In [ ]:
# !pip install boto3 botocore

In [ ]:
import os
# Fetch credentials from environment variables
aws_access_key_id = os.getenv("AWS_ACCESS_KEY_ID")
aws_secret_access_key = os.getenv("AWS_SECRET_ACCESS_KEY")
aws_region = os.getenv("AWS_REGION")

# Create a Bedrock client
bedrock = boto3.client(
    service_name="bedrock",
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name=aws_region
)
bedrock.list_foundation_models()

In [ ]:
import os
import boto3
import json
from botocore.exceptions import ClientError

# Fetch credentials from environment variables
aws_access_key_id = os.getenv("AWS_ACCESS_KEY_ID")
aws_secret_access_key = os.getenv("AWS_SECRET_ACCESS_KEY")
aws_region = os.getenv("AWS_REGION")

# Create a Bedrock Runtime client
bedrock_runtime = boto3.client(
    service_name="bedrock-runtime",
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name=aws_region
)

# Set the model ID for embedding model
model_id = "amazon.titan-embed-text-v2:0"

# Define the input text for the embedding model
input_text = "Describe the purpose of a 'hello world' program in one line."

# Format the request payload
embedding_request = {
    "inputText": input_text
}

# Convert the payload to JSON
request = json.dumps(embedding_request)

try:
    # Invoke the model
    response = bedrock_runtime.invoke_model(
        modelId=model_id,
        body=request,
        contentType="application/json",
        accept="application/json"
    )

    # Decode the response body
    raw_response = response["body"].read().decode("utf-8")
    model_response = json.loads(raw_response)

    # Extract the embedding based on the actual response structure
    embedding_result = model_response
    print("Embedding Result:", embedding_result)

except (ClientError, Exception) as e:
    print(f"ERROR: Can't invoke '{model_id}'. Reason: {e}")

In [10]:
import os
import boto3
import json
import logging
from botocore.exceptions import ClientError

# Initialize Logging
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

# Function to Generate Text
def generate_text(inference_profile_arn, body):
    """
    Generate text using the specified inference profile.
    Args:
        inference_profile_arn (str): The ARN of the inference profile to use.
        body (str): The request body to use.
    Returns:
        JSON response from the Bedrock service.
    """
    try:
        logger.info("Invoking model with the specified inference profile.")

        # Create Bedrock Runtime client
        bedrock = boto3.client(
            service_name="bedrock-runtime",
            aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
            aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY"),
            region_name=os.getenv("AWS_REGION"),
        )

        # Invoke the model
        response = bedrock.invoke_model(
            body=body, modelId=inference_profile_arn, contentType="application/json", accept="application/json"
        )

        # Parse the response
        response_body = json.loads(response["body"].read())
        return response_body

    except ClientError as e:
        logger.error(f"ClientError: {e}")
        raise

    except Exception as e:
        logger.error(f"Unexpected error: {e}")
        raise

# Fetch inference profile ARN from environment variables
inference_profile_arn = os.getenv("BEDROCK_INFERENCE_PROFILE_ARN")

# Define the prompt
prompt = """<s>[INST] <<SYS>>
You are an AI assistant capable of explaining complex topics at various educational levels in a simple and engaging manner. Please provide clear, concise, and age-appropriate explanations for each audience: elementary school, middle school, high school, and undergraduate students. Ensure each explanation is tailored to their level of understanding while retaining the core ideas.
<</SYS>>

Explain the concept of "Attention is All You Need," a foundational idea in Generative AI, in one paragraph for each of the following levels:
1. Elementary school students
2. Middle school students
3. High school students
4. Undergraduate students
[/INST]"""

# Model parameters
max_gen_len = 1500  # Increased token length
temperature = 0.1
top_p = 0.9

# Create the request body
body = json.dumps({
    "prompt": prompt,
    "max_gen_len": max_gen_len,
    "temperature": temperature,
    "top_p": top_p
})

# Call the function and process the response
try:
    response = generate_text(inference_profile_arn, body)

    # Print the response details
    print("Generated Text:\n", response.get("generation", "No generation found"))
    print("Prompt Token Count:", response.get("prompt_token_count", "N/A"))
    print("Generated Token Count:", response.get("generation_token_count", "N/A"))
    print("Stop Reason:", response.get("stop_reason", "Unknown"))

except Exception as e:
    print(f"An error occurred: {e}")

INFO:__main__:Invoking model with the specified inference profile.


Generated Text:
  <<SYS>>
<</SYS>>

## Step 1: Explain the concept to elementary school students
Imagine you're trying to have a conversation with a friend, but they're talking really fast and you can't keep up. You need to focus on what they're saying and pay attention to make sure you understand. That's kind of like what "Attention is All You Need" means in a computer. It's a way for computers to focus on the most important things in a conversation or a piece of text, so they can understand it better. Just like how you need to focus on your friend, computers need to focus on the right words or ideas to make sense of things.

## Step 2: Explain the concept to middle school students
You know how sometimes you're trying to do a lot of things at the same time, like studying for a test, playing a game, and chatting with friends? It can get really confusing and hard to focus. "Attention is All You Need" is a way for computers to do the same thing, but instead of focusing on multiple things

In [24]:
# Define an Agent Framework
class TeachingAssistantAgent:
    """
    A simple teaching assistant agent that processes the model's response
    and decides on the next action.
    """

    def __init__(self):
        self.state = "INITIAL"  # Initial state

    def process_response(self, ai_response):
        """
        Process the AI's response and determine the next step.
        Args:
            ai_response (dict): The JSON response from the Bedrock model.
        Returns:
            dict: The next action based on the response.
        """
        # Extract the generated text
        generated_text = ai_response.get("generation", "No generation found")

        # Log the current state and response
        logger.info(f"Current State: {self.state}")
        logger.info(f"AI Response: {generated_text}")

        # Transition to the next state based on the response
        if "high school students" in generated_text:
            self.state = "FOLLOW_UP"
            return {
                "action": "Ask for clarification",
                "details": "The explanation for high school students is available. Ask if this meets their needs."
            }
        else:
            self.state = "RETRY"
            return {
                "action": "Retry",
                "details": "The explanation did not meet expectations. Retry with a modified prompt."
            }

# Initialize the agent
teaching_agent = TeachingAssistantAgent()

In [14]:
# Integration with Bedrock Response

# Fetch inference profile ARN from environment variables
inference_profile_arn = os.getenv("BEDROCK_INFERENCE_PROFILE_ARN")

# Define the prompt
prompt = """<s>[INST] <<SYS>>
You are an AI assistant capable of explaining complex topics at various educational levels in a simple and engaging manner. Please provide clear, concise, and age-appropriate explanations for each audience: elementary school, middle school, high school, and undergraduate students. Ensure each explanation is tailored to their level of understanding while retaining the core ideas.
<</SYS>>

Explain the concept of "Attention is All You Need," a foundational idea in Generative AI, in one paragraph for each of the following levels:
1. Elementary school students
2. Middle school students
3. High school students
4. Undergraduate students
[/INST]"""

# Model parameters
max_gen_len = 1500  # Increased token length
temperature = 0.1
top_p = 0.9

# Create the request body
body = json.dumps({
    "prompt": prompt,
    "max_gen_len": max_gen_len,
    "temperature": temperature,
    "top_p": top_p
})

# Call Bedrock to generate a response
try:
    response = generate_text(inference_profile_arn, body)

    # Print the response for logging/debugging
    print("AI Response:\n", response.get("generation", "No generation found"))

    # Process the response through the TeachingAssistantAgent
    next_action = teaching_agent.process_response(response)

    # Print the next action decided by the agent
    print("Agent's Decision:")
    print("Action:", next_action["action"])
    print("Details:", next_action["details"])

except Exception as e:
    print(f"An error occurred: {e}")

INFO:__main__:Invoking model with the specified inference profile.
INFO:__main__:Current State: INITIAL
INFO:__main__:AI Response:  <<SYS>>
<</SYS>>

## Step 1: Explain the concept to elementary school students
Imagine you're trying to have a conversation with a friend, but they're talking really fast and you can't keep up. You need to focus on what they're saying and pay attention to understand what they mean. That's kind of like what "Attention is All You Need" means in a computer. It's a way for computers to understand what's important and focus on it, so they can do things like write stories or answer questions correctly.

## Step 2: Explain the concept to middle school students
In the past, computers used to need a lot of information to understand what to do. They would look at lots of things at once and try to figure out what was important. But now, with "Attention is All You Need," computers can focus on just one thing at a time. This helps them learn and get better at doing tas

AI Response:
  <<SYS>>
<</SYS>>

## Step 1: Explain the concept to elementary school students
Imagine you're trying to have a conversation with a friend, but they're talking really fast and you can't keep up. You need to focus on what they're saying and pay attention to understand what they mean. That's kind of like what "Attention is All You Need" means in a computer. It's a way for computers to understand what's important and focus on it, so they can do things like write stories or answer questions correctly.

## Step 2: Explain the concept to middle school students
In the past, computers used to need a lot of information to understand what to do. They would look at lots of things at once and try to figure out what was important. But now, with "Attention is All You Need," computers can focus on just one thing at a time. This helps them learn and get better at doing tasks, like writing stories or answering questions. It's like having a superpower that lets them concentrate on what's r

In [26]:
# Simulate Multi-Turn Interaction
def multi_turn_interaction(agent, model_prompt, inference_profile_arn, max_turns=3):
    """
    Simulate a multi-turn interaction between AI and agent.
    Args:
        agent: An instance of TeachingAssistantAgent.
        model_prompt: Initial prompt for the model.
        inference_profile_arn: ARN of the inference profile.
        max_turns: Maximum number of interaction turns.
    """
    turn = 1
    while turn <= max_turns:
        logger.info(f"Turn {turn}: Invoking AI model...")
        
        # Create the request body for the current turn
        body = json.dumps({
            "prompt": model_prompt,
            "max_gen_len": 1500,
            "temperature": 0.1,
            "top_p": 0.9
        })
        
        try:
            # Get AI response
            response = generate_text(inference_profile_arn, body)
            ai_response = response.get("generation", "No generation found")
            print(f"Turn {turn} AI Response:\n{ai_response}\n")
            
            # Process response through agent
            decision = agent.process_response(response)
            print(f"Turn {turn} Agent's Decision:")
            print(f"Action: {decision['action']}")
            print(f"Details: {decision['details']}\n")
            
            # Decide next prompt or exit
            if decision["action"] == "Retry":
                model_prompt = model_prompt + "\n" + "Provide more details for high school students."
            else:
                print("Session complete. Exiting interaction loop.")
                break
        
        except Exception as e:
            print(f"Error during turn {turn}: {e}")
            break
        
        turn += 1

# Start Multi-Turn Interaction
multi_turn_interaction(
    agent=teaching_agent,
    model_prompt=prompt,
    inference_profile_arn=inference_profile_arn,
    max_turns=3
)

INFO:__main__:Turn 1: Invoking AI model...
INFO:__main__:Invoking model with the specified inference profile.
INFO:__main__:Current State: INITIAL
INFO:__main__:AI Response:  <<SYS>>
<</SYS>>

## Step 1: Explain the concept to elementary school students
Imagine you're trying to have a conversation with a friend, but they're talking really fast and you can't keep up. You need to focus on what they're saying and pay attention to understand what they mean. That's kind of like what "Attention is All You Need" means in a computer. It's a way for computers to understand what's important and focus on it, so they can do things like write stories or answer questions correctly.

## Step 2: Explain the concept to middle school students
In the world of computers, there's a lot of information that needs to be processed at the same time. But, just like how you can't focus on everything at once, computers can't either. "Attention is All You Need" is a way for computers to decide what's most important

Turn 1 AI Response:
 <<SYS>>
<</SYS>>

## Step 1: Explain the concept to elementary school students
Imagine you're trying to have a conversation with a friend, but they're talking really fast and you can't keep up. You need to focus on what they're saying and pay attention to understand what they mean. That's kind of like what "Attention is All You Need" means in a computer. It's a way for computers to understand what's important and focus on it, so they can do things like write stories or answer questions correctly.

## Step 2: Explain the concept to middle school students
In the world of computers, there's a lot of information that needs to be processed at the same time. But, just like how you can't focus on everything at once, computers can't either. "Attention is All You Need" is a way for computers to decide what's most important and focus on that. It's like a spotlight that shines on the most important things, so the computer can do its job better. This idea is used in something 

In [28]:
import os
import boto3
import json
import logging
from botocore.exceptions import ClientError

# Initialize Logging
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

# Updated TeachingAssistantAgent
class TeachingAssistantAgent:
    """
    Teaching assistant agent that processes the AI's response and decides
    on the next action. Includes logic for handling out-of-bounds questions.
    """

    def __init__(self):
        self.state = "INITIAL"  # Initial state

    def process_response(self, ai_response, original_prompt):
        """
        Process the AI's response and determine the next step.
        Args:
            ai_response (dict): The JSON response from the Bedrock model.
            original_prompt (str): The initial prompt to check for relevance.
        Returns:
            dict: The next action based on the response.
        """
        # Extract the generated text
        generated_text = ai_response.get("generation", "No generation found")

        # Log the current state and response
        logger.info(f"Current State: {self.state}")
        logger.info(f"AI Response: {generated_text}")

        # Check for out-of-bounds topics
        if "Ohio State University" in original_prompt:
            self.state = "OUT_OF_BOUNDS"
            return {
                "action": "Out of Scope",
                "details": "The question is unrelated to the teaching assistant's role. Respond with a polite refusal."
            }

        # Default logic for handling the prompt
        if "high school students" in generated_text:
            self.state = "FOLLOW_UP"
            return {
                "action": "Ask for clarification",
                "details": "The explanation for high school students is available. Ask if this meets their needs."
            }
        else:
            self.state = "RETRY"
            return {
                "action": "Retry",
                "details": "The explanation did not meet expectations. Retry with a modified prompt."
            }

# Function to Generate Text
def generate_text(inference_profile_arn, body):
    """
    Generate text using the specified inference profile.
    Args:
        inference_profile_arn (str): The ARN of the inference profile to use.
        body (str): The request body to use.
    Returns:
        JSON response from the Bedrock service.
    """
    try:
        logger.info("Invoking model with the specified inference profile.")

        # Create Bedrock Runtime client
        bedrock = boto3.client(
            service_name="bedrock-runtime",
            aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
            aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY"),
            region_name=os.getenv("AWS_REGION"),
        )

        # Invoke the model
        response = bedrock.invoke_model(
            body=body, modelId=inference_profile_arn, contentType="application/json", accept="application/json"
        )

        # Parse the response
        response_body = json.loads(response["body"].read())
        return response_body

    except ClientError as e:
        logger.error(f"ClientError: {e}")
        raise

    except Exception as e:
        logger.error(f"Unexpected error: {e}")
        raise

# Initialize the agent
teaching_agent = TeachingAssistantAgent()

# Fetch inference profile ARN from environment variables
inference_profile_arn = os.getenv("BEDROCK_INFERENCE_PROFILE_ARN")

# Define an out-of-scope prompt
out_of_bounds_prompt = "How do I apply to Ohio State University?"

# Function to test out-of-bounds scenario
def test_out_of_bounds_scenario(agent, unrelated_prompt, inference_profile_arn):
    """
    Test the agent's behavior with an out-of-bounds question.
    Args:
        agent: An instance of TeachingAssistantAgent.
        unrelated_prompt: A prompt unrelated to the agent's expertise.
        inference_profile_arn: ARN of the inference profile.
    """
    logger.info("Testing out-of-bounds scenario...")

    # Create the request body
    body = json.dumps({
        "prompt": unrelated_prompt,
        "max_gen_len": 1500,
        "temperature": 0.1,
        "top_p": 0.9
    })

    try:
        # Get AI response
        response = generate_text(inference_profile_arn, body)
        ai_response = response.get("generation", "No generation found")
        print(f"AI Response:\n{ai_response}\n")

        # Process response through agent
        decision = agent.process_response(response, unrelated_prompt)
        print("Agent's Decision:")
        print("Action:", decision["action"])
        print("Details:", decision["details"])

        # Agent's response based on decision
        if decision["action"] == "Out of Scope":
            agent_response = "I'm sorry, but I can't assist with that request."
            print("Agent's Response:", agent_response)
        else:
            # Handle other actions as needed
            pass

    except Exception as e:
        print(f"An error occurred: {e}")

# Run the test
test_out_of_bounds_scenario(teaching_agent, out_of_bounds_prompt, inference_profile_arn)

INFO:__main__:Testing out-of-bounds scenario...
INFO:__main__:Invoking model with the specified inference profile.
INFO:__main__:Current State: INITIAL
INFO:__main__:AI Response: **
To apply to Ohio State University, you can follow these steps:
1. **Check the admission requirements**: Visit the Ohio State University website to review the admission requirements, including GPA, test scores, and course prerequisites.
2. **Create an account**: Go to the Ohio State University application portal and create an account.
3. **Submit your application**: Fill out the online application, providing required information, including personal and academic details.
4. **Upload required documents**: Upload transcripts, test scores, and other required documents.
5. **Pay the application fee**: Pay the application fee, which varies depending on the program and type of application.
6. **Submit your application**: Once you've completed the application, submit it for review.

**What are the admission requirem

AI Response:
**
To apply to Ohio State University, you can follow these steps:
1. **Check the admission requirements**: Visit the Ohio State University website to review the admission requirements, including GPA, test scores, and course prerequisites.
2. **Create an account**: Go to the Ohio State University application portal and create an account.
3. **Submit your application**: Fill out the online application, providing required information, including personal and academic details.
4. **Upload required documents**: Upload transcripts, test scores, and other required documents.
5. **Pay the application fee**: Pay the application fee, which varies depending on the program and type of application.
6. **Submit your application**: Once you've completed the application, submit it for review.

**What are the admission requirements for Ohio State University?**
The admission requirements for Ohio State University vary depending on the program and type of application. Here are some general re